# BioMedBERT: Process BREATHE - June 21

In [46]:
# imports
import os
import json
import sys
import nltk
import glob
import numpy as np
import pandas as pd
import textwrap
import tensorflow as tf
from google.cloud import storage

### Preprocess text
Remove punсtuation, uppercase letters and non-utf symbols.

In [47]:
regex_tokenizer = nltk.RegexpTokenizer("\w+")

def normalize_text(text):
    # lowercase text
    text = str(text).lower()
    # remove non-UTF
    text = text.encode("utf-8", "ignore").decode()
    # remove punctuation symbols
    text = " ".join(regex_tokenizer.tokenize(text))
    return text

## Create Expanded csv dataset

In [48]:
project_id = 'ai-vs-covid19'
storage_client = storage.Client(project=project_id)
bucket = 'breathe-dump'

### Get 2020-06-18 dump

In [49]:
# !gsutil -m rm -rf gs://ekaba-assets/full_body/

In [50]:
bucket+'/v1/2020-06-18'

'breathe-dump/v1/2020-06-18'

In [51]:
bucket=storage_client.get_bucket(bucket)
# List all objects that satisfy the filter.
blobs=bucket.list_blobs(prefix='v1/2020-06-18')

In [52]:
blob = [blob for blob in blobs]

In [53]:
len(blob)

119

In [54]:
# Create a function called "chunks" with two arguments, l and n:
def split_list(data, chunk):
    # For item i in a range that is a length of data (l),
    for i in range(0, len(data), chunk):
        # Create an index range for data of chunk (e.g. 5) items:
        yield data[i:i+chunk]

In [25]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
udev             40G     0   40G   0% /dev
tmpfs           8.0G   19M  8.0G   1% /run
/dev/sda1       394G  139G  240G  37% /
tmpfs            40G     0   40G   0% /dev/shm
tmpfs           5.0M     0  5.0M   0% /run/lock
tmpfs            40G     0   40G   0% /sys/fs/cgroup
tmpfs           8.0G     0  8.0G   0% /run/user/308194119


In [26]:
blob[0:5]

[<Blob: breathe-dump, v1/2020-06-18/arxiv/000000000000.csv, 1592475725719014>,
 <Blob: breathe-dump, v1/2020-06-18/bioasq/000000000000.csv, 1592475779299830>,
 <Blob: breathe-dump, v1/2020-06-18/bioasq/000000000001.csv, 1592475787152157>,
 <Blob: breathe-dump, v1/2020-06-18/bioasq/000000000002.csv, 1592475777874489>,
 <Blob: breathe-dump, v1/2020-06-18/bioasq/000000000003.csv, 1592475781637700>]

In [29]:
def download_to_local(folder, blob_lst):
    print('File download Started…. Wait for the job to complete.')
    # Create this folder locally if not exists
    if not os.path.exists(folder):
        os.makedirs(folder)
    # Iterating through for loop one by one using API call
    for blob in blob_lst:
        if not os.path.exists(os.path.join(folder, (blob.name).split('/')[-2])):
            os.makedirs(os.path.join(folder, (blob.name).split('/')[-2]))
            
        destination_uri = '{}/{}/{}'.format(folder, (blob.name).split('/')[-2], (blob.name).split('/')[-1])
        blob.download_to_filename(destination_uri)
        print('Exported {} to {}'.format(blob.name, destination_uri))

In [45]:
# download csv's
download_to_local('BREATHE-June-21', blob)

In [43]:
# make combined csv
def combined_csv(data_folder):
    all_filenames = [i for i in glob.glob('{}/*/*'.format(data_folder))]
    #combine all files in the list
    combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
    return combined_csv

In [44]:
# blob_csv_A = combined_csv('data')
blob_csv = combined_csv('BREATHE-June-21')

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (3,6) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


KeyboardInterrupt: 

In [ ]:
# len(blob_csv)

In [89]:
blob_csv.to_csv('gs://ekaba-assets/full_body_text.csv')

/usr/local/lib/python3.5/dist-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [ ]:
# blob_csv_A.to_csv( "ncbi_comm_use_csv_A.csv", index=False, encoding='utf-8-sig')
# blob_csv.to_csv( "ncbi_comm_use.csv", index=False, encoding='utf-8-sig')

In [ ]:
# copy files from gcs bucket
# !gsutil -m cp gs://ekaba-assets/ncbi_comm_use_BODY.csv .

In [ ]:
# body = pd.read_csv('ncbi_comm_use.csv')

In [90]:
body_sel = blob_csv[['full_body']]

In [92]:
# body_sel.head()

In [93]:
body_sel.to_csv('gs://ekaba-assets/full_body_text_BODY.csv')

In [95]:
body_sel.to_csv( "full_body_text_BODY.csv", index=False, encoding='utf-8-sig')

In [101]:
# remove FULL ncbi_comm_use.csv
!rm full_body_text_BODY.csv

In [96]:
# convert csv to txt
import csv
import sys
maxInt = sys.maxsize
csv.field_size_limit(maxInt)

csv_file = 'full_body_text_BODY.csv'
txt_file = 'full_body_text_BODY.txt'
with open(txt_file, "w") as my_output_file:
    with open(csv_file, "r") as my_input_file:
        [ my_output_file.write(" ".join(row)+'\n') for row in csv.reader(my_input_file)]
    my_output_file.close()

In [97]:
# move text file to GCS
# !gsutil -m cp ncbi_comm_use_BODY.txt gs://ekaba-assets/

In [98]:
# remove csv file
# !rm ncbi_comm_use_BODY.csv

In [99]:
from tensorflow.keras.utils import Progbar
def count_lines(filename):
    count = 0
    with open(filename) as fi:
        for line in fi:
            count += 1
    return count

In [102]:
# Apply normalization to entire dataset
RAW_DATA_FPATH = "full_body_text_BODY.txt"
PRC_DATA_FPATH = "processed_full_body_text_BODY.txt"

# apply normalization to the dataset

total_lines = count_lines(RAW_DATA_FPATH)
bar = Progbar(total_lines)

with open(RAW_DATA_FPATH,encoding="utf-8") as fi:
  with open(PRC_DATA_FPATH, "w",encoding="utf-8") as fo:
    for l in fi:
      fo.write(normalize_text(l)+"\n")
      bar.add(1)

2356953/2356953 [==============================] - 5317s 2ms/step


In [103]:
# move processed text file to GCS
!gsutil -m cp processed_full_body_text_BODY.txt gs://ekaba-assets/

Copying file://processed_full_body_text_BODY.txt [Content-Type=text/plain]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

| [1/1 files][ 61.4 GiB/ 61.4 GiB] 100% Done  56.6 MiB/s ETA 00:00:00           
Operation completed over 1 objects/61.4 GiB.                                     


In [105]:
# remove intermediate files
!rm processed_full_body_text_BODY.txt #full_body_text_BODY.txt #processed_ncbi_comm_use_BODY.txt